# CCSD theory for a closed-shell reference

In this notebook we will use Wick\&d to generate and implement equations for the spin-integrated CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers_spin_integrated.py`.
In this example, we run a CCSD computation on the H<sub>2</sub>O molecule, reading all the relevant information from the file `sr-h2o-cc-pvdz.npy`.

In [1]:
import time
import wicked as w
import numpy as np
from examples_helpers_spin_integrated import *
import itertools

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h2o-cc-pvdz.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [13]:
molecule = "sr-h2o-cc-pvdz"
with open(f"{molecule}.npy", "rb") as f:
    Eref = np.load(f)
    Ecorr_ref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()
invD = compute_inverse_denominators(H, nocc, nvir, 3)

## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

To do spin-integration, we simply introduce separate orbital spaces for alpha and beta spinorbitals. We use the convention that small cap letters denote alpha spinorbitals and big cap letters denote beta spinorbitals.

We include all the $M_S$-conserving operators in $\hat{T}$ and $\hat{H}$. We use the optional argument `unique=True` in the convenience function `w.op` to prevent combinatorically equivalent operators from being included more than once, which would result in incorrect factors.

In [14]:
w.reset_space()
# alpha
w.add_space('o', 'fermion', 'occupied', list('ijklmn'))
w.add_space('v', 'fermion', 'unoccupied', list('abcdef'))
# beta
w.add_space('O', 'fermion', 'occupied', list('IJKLMN'))
w.add_space('V', 'fermion', 'unoccupied', list('ABCDEF'))

Top = w.op("T", ["v+ o", "V+ O", "v+ v+ o o", "V+ V+ O O", "V+ v+ O o", "v+ v+ v+ o o o", "V+ V+ V+ O O O", "v+ v+ V+ o o O", "v+ V+ V+ o O O"], unique=True)
Hops = []
for i in itertools.product(['v+', 'o+'],['v', 'o']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'O+'],['V', 'O']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'o+'],['v+', 'o+'], ['v', 'o'], ['v', 'o']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'O+'],['V+', 'O+'], ['V', 'O'], ['V', 'O']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'o+'],['V+', 'O+'], ['v', 'o'], ['V', 'O']):
    Hops.append(' '.join(i))
Hop = w.op("H", Hops, unique=True)


In [4]:
Hbar = w.bch_series(Hop, Top, 4)

In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [5]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 6)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [11]:
mbeq.keys()

dict_keys(['OOO|VOO', 'OOO|VVO', 'OOO|VVV', 'OOV|VOO', 'OOV|VVO', 'OOV|VVV', 'OO|OO', 'OO|VO', 'OO|VV', 'OVV|VVO', 'OVV|VVV', 'OV|OO', 'OV|VO', 'OV|VV', 'O|O', 'O|V', 'VV|OO', 'VV|VO', 'VV|VV', 'V|O', 'V|V', 'oOO|OOv', 'oOO|VOo', 'oOO|VOv', 'oOO|VVo', 'oOO|VVv', 'oOV|OOv', 'oOV|VOo', 'oOV|VOv', 'oOV|VVo', 'oOV|VVv', 'oO|Oo', 'oO|Ov', 'oO|Vo', 'oO|Vv', 'oVV|VOv', 'oVV|VVv', 'oV|Oo', 'oV|Ov', 'oV|Vo', 'oV|Vv', 'ooO|Ovo', 'ooO|Ovv', 'ooO|Voo', 'ooO|Vvo', 'ooO|Vvv', 'ooV|Ovo', 'ooV|Ovv', 'ooV|Vvo', 'ooV|Vvv', 'ooo|voo', 'ooo|vvo', 'ooo|vvv', 'oov|voo', 'oov|vvo', 'oov|vvv', 'oo|oo', 'oo|vo', 'oo|vv', 'ovO|Ovo', 'ovO|Ovv', 'ovO|Voo', 'ovO|Vvo', 'ovO|Vvv', 'ovV|Ovv', 'ovV|Vvo', 'ovV|Vvv', 'ovv|vvo', 'ovv|vvv', 'ov|oo', 'ov|vo', 'ov|vv', 'o|o', 'o|v', 'vOO|VOo', 'vOO|VOv', 'vOO|VVo', 'vOO|VVv', 'vOV|VOv', 'vOV|VVo', 'vOV|VVv', 'vO|Oo', 'vO|Ov', 'vO|Vo', 'vO|Vv', 'vV|Oo', 'vV|Ov', 'vV|Vo', 'vV|Vv', 'vvO|Vvo', 'vvO|Vvv', 'vv|oo', 'vv|vo', 'vv|vv', 'v|o', 'v|v', '|'])

In [8]:
energy_eq = generate_equation(mbeq, 0, 0, '|')
t1_aa_eq = generate_equation(mbeq, 1, 1, 'o|v')
t1_bb_eq = generate_equation(mbeq, 1, 1, 'O|V')
t2_aaaa_eq = generate_equation(mbeq, 2, 2, 'oo|vv')
t2_bbbb_eq = generate_equation(mbeq, 2, 2, 'OO|VV')
t2_abab_eq = generate_equation(mbeq, 2, 2, 'oO|Vv')
t3_aaaaaa_eq = generate_equation(mbeq, 3, 3, 'ooo|vvv')
t3_bbbbbb_eq = generate_equation(mbeq, 3, 3, 'OOO|VVV')
t3_abbabb_eq = generate_equation(mbeq, 3, 3, 'oOO|VVv')
t3_aabaab_eq = generate_equation(mbeq, 3, 3, 'ooO|Vvv')
exec(energy_eq)
exec(t1_aa_eq)
exec(t1_bb_eq)
exec(t2_aaaa_eq)
exec(t2_bbbb_eq)
exec(t2_abab_eq)
exec(t3_aaaaaa_eq)
exec(t3_bbbbbb_eq)
exec(t3_abbabb_eq)
exec(t3_aabaab_eq)

## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [15]:
T = {"ov": np.zeros((nocc, nvir)), "OV": np.zeros((nocc,nvir)),\
    "oovv": np.zeros((nocc, nocc, nvir, nvir)), "OOVV": np.zeros((nocc, nocc, nvir, nvir)), "oOvV": np.zeros((nocc, nocc, nvir, nvir)),\
    "ooovvv": np.zeros((nocc, nocc, nocc, nvir, nvir, nvir)), "OOOVVV": np.zeros((nocc, nocc, nocc, nvir, nvir, nvir)),\
    "ooOvvV": np.zeros((nocc, nocc, nocc, nvir, nvir, nvir)), "oOOvVV": np.zeros((nocc, nocc, nocc, nvir, nvir, nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.time()
Ecorr_ref = -0.22800600 # from ccpy

maxiter = 50

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_(H, T)
    Etot_w = Eref + Ecorr_w
    R["ov"] = evaluate_residual_ov(H, T)
    R["OV"] = evaluate_residual_OV(H, T)
    Roovv = evaluate_residual_oovv(H, T)
    R["oovv"] = antisymmetrize_residual_2_2(Roovv, nocc, nvir)
    ROOVV = evaluate_residual_OOVV(H, T)
    R["OOVV"] = antisymmetrize_residual_2_2(ROOVV, nocc, nvir)
    R["oOvV"] = evaluate_residual_oOvV(H, T) # no need to antisymmetrize
    Rooovvv = evaluate_residual_ooovvv(H, T)
    R["ooovvv"] = antisymmetrize_residual_3_3(Rooovvv, nocc, nvir)
    ROOOVVV = evaluate_residual_OOOVVV(H, T)
    R["OOOVVV"] = antisymmetrize_residual_3_3(ROOOVVV, nocc, nvir)
    RooOvvV = evaluate_residual_ooOvvV(H, T)
    R["ooOvvV"] = antisymmetrize_residual_3_3_aab(RooOvvV, nocc, nvir)
    RoOOvVV = evaluate_residual_oOOvVV(H, T)
    R["oOOvVV"] = antisymmetrize_residual_3_3_abb(RoOOvVV, nocc, nvir)

    # 2. amplitude update
    update_cc_amplitudes(T, R, invD, 3)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R["ov"]) ** 2 + np.linalg.norm(R["OV"]) ** 2 + \
                    np.linalg.norm(R["oovv"]) ** 2 + np.linalg.norm(R["OOVV"]) ** 2 + np.linalg.norm(R["oOvV"]) ** 2 + \
                    np.linalg.norm(R["ooovvv"]) ** 2 + np.linalg.norm(R["OOOVVV"]) ** 2 + np.linalg.norm(R["ooOvvV"]) ** 2 + np.linalg.norm(R["oOOvVV"]) ** 2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}")
    if norm_R < 1.0e-8:
        break

end = time.time()
t = end - start

print("-" * len(header))
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:.4f} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)


-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -75.989795787487    +0.000000000000    1.299100e+00
  1    -76.204143461766    -0.214347674279    3.759346e-01
  2    -76.207264973707    -0.217469186220    7.975060e-02
  3    -76.215394719697    -0.225598932210    3.650329e-02
  4    -76.216352488277    -0.226556700790    1.574033e-02
  5    -76.217257710823    -0.227461923336    8.037161e-03
  6    -76.217497462265    -0.227701674778    4.115558e-03
  7    -76.217663745121    -0.227867957634    2.262742e-03
  8    -76.217724934713    -0.227929147226    1.253725e-03
  9    -76.217763354631    -0.227967567144    7.258450e-04
 10    -76.217780155499    -0.227984368012    4.219787e-04
 11    -76.217790274780    -0.227994487293    2.521932e-04
 12    -76.217795220037    -0.227999432550    1.509653e-04
 13    -76.217798150031    -0.228002362545    9.18487

### (Optional) Integral generation
For completeness, we document here how `sr-h2o-cc-pvdz.npy` was generated. You will need to install PySCF to run the following cell. Due to the size, we do not provide the cc-pVTZ integrals in the repository, but you can generate it with the cell below and test the speed-up compared to the spinorbital code in the other notebook.

In [8]:
import pyscf, pyscf.cc, pyscf.mp

mol = pyscf.gto.M(atom="""
O 
H 1 1.1
H 1 1.1 2 104
""", basis='cc-pvdz')

mf = pyscf.scf.RHF(mol)
_ = mf.kernel()
cc = pyscf.cc.CCSD(mf)
_ = cc.kernel()
mp = pyscf.mp.MP2(mf)
_ = mp.kernel()

nocc = mol.nelectron // 2
nvir = mol.nao - nocc

eri = pyscf.ao2mo.full(mol.intor('int2e'), mf.mo_coeff)
eri = eri.swapaxes(1, 2)

blocks = get_index_blocks(energy_eq+t1_aa_eq+t1_bb_eq+t2_aaaa_eq+t2_bbbb_eq+t2_abab_eq)

V = eri
V_asym = eri - eri.swapaxes(2, 3)
F = np.diag(mf.mo_energy)
sl = {'o': slice(0, nocc), 'v': slice(nocc, nocc+nvir)}
H = {}
for block in blocks:
    if len(block) == 2:
        H[block] = F[sl[block[0].lower()], sl[block[1].lower()]]
    else:
        abab = (block[0].islower() + block[1].islower() + block[2].islower() + block[3].islower() == 2)
        if abab:
            H[block] = V[sl[block[0].lower()], sl[block[1].lower()], sl[block[2].lower()], sl[block[3].lower()]]
        else:
            H[block] = V_asym[sl[block[0].lower()], sl[block[1].lower()], sl[block[2].lower()], sl[block[3].lower()]]

with open('sr-h2o-cc-pvdz.npy', 'wb') as f:
    np.save(f, mf.e_tot)
    np.save(f, cc.e_corr)
    np.save(f, (nocc,nvir))
    np.save(f, H, allow_pickle=True)

converged SCF energy = -75.9897957874868
E(CCSD) = -76.21370580011651  E_corr = -0.2239100126297478
E(MP2) = -76.2041434617655  E_corr = -0.214347674278744
E(SCS-MP2) = -76.2010219265648  E_corr = -0.211226139078036
